In [1]:
!pip install langchain langchain-classic langchain-community langchain-huggingface langchain-unstructured unstructured unstructured-client "unstructured[html]" chromadb langchain-unstructured unstructured-client unstructured "unstructured[pdf]" python-magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.6/219.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!apt-get update
!apt-get install -y poppler-utils

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,572 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Get:14 http:/

In [3]:
!unzip chroma_db.zip -d /content/chroma_db

Archive:  chroma_db.zip
   creating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/
  inflating: /content/chroma_db/chroma.sqlite3  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/index_metadata.pickle  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/link_lists.bin  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/header.bin  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/length.bin  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/data_level0.bin  


In [4]:
import requests
from bs4 import BeautifulSoup , Comment
import tempfile
import os
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_community.vectorstores import Chroma
import re

# Define the embedding model

In [5]:
from google.colab import userdata

embedding_model = HuggingFaceEndpointEmbeddings(
    model = 'sentence-transformers/all-MiniLM-L6-v2',
    huggingfacehub_api_token=userdata.get('HF_TOKEN')
)

# define vector store

In [6]:
vector_store = Chroma(
  embedding_function=embedding_model,
  persist_directory="chroma_db",
  collection_name="collection_research_guide"
)

/tmp/ipython-input-1021428914.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_store = Chroma(


# Get structured document from pdf using UnstructedPDFLoader

In [11]:
def get_structured_docs(docs , url):

  # Prepare structured document array
  structured_docs = []

  current_title = None
  current_header = None

  # Categories to treat as titles
  TITLE_CATEGORIES = { "title", "pagetitle", "main title" }

  # Categories to treat as headers
  HEADER_CATEGORIES = {"header", "header1", "header2", "header3", "header4", "header5", "header6", "subheading"}

  # Categories to ignore completely
  IGNORE_CATEGORIES = {"figure", "image" , "uncategorizedtext" , "footer"}

  for d in docs:
      category = d.metadata.get("category", "").lower()
      text = d.page_content.strip()

      if not text:
          continue

      if category in TITLE_CATEGORIES:
          current_title = text
          current_header = None  # reset header when new title
      elif category in HEADER_CATEGORIES:
          current_header = text
      elif category in IGNORE_CATEGORIES:
          continue  # skip figures/images
      else:
          urls = " \n ".join(url)
          # Other categories: paragraph, list, table, link, code, quote, etc.
          structured_docs.append(
              Document(
                  page_content=f" Title-{current_title or "Not available"} , heading-{current_header or "Not available"} , url-{urls or "Not available"} \n\n description - {text}",
                  metadata={
                      "title": current_title or "Not available",
                      "header": current_header or "Not available",
                      "url": urls
                  }
              )
          )

  return structured_docs

# Load pdf using unstructured pdf loader

In [12]:
from langchain_community.document_loaders import UnstructuredPDFLoader

file_paths = "/content/MMTTC - IIT(ISM) Dhanbad.pdf"

loader = UnstructuredPDFLoader(file_paths , mode="elements", strategy="hi_res" )
docs = loader.load()

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

In [15]:
c = set()

for doc in docs :
  c.add(doc.metadata["category"])

  if doc.metadata["category"] == "UncategorizedText":
    print(doc.page_content)

c

S. No.
Technologies: Bridging Science and Practice
S. No.


{'Footer', 'Image', 'ListItem', 'Table', 'Title', 'UncategorizedText'}

In [16]:
structured_docs = get_structured_docs(docs , ['https://people.iitism.ac.in/~download/MalviyaMission/MMTTC%20-%20IIT(ISM)%20Dhanbad.pdf' , 'https://www.iitism.ac.in/center'])

structured_docs

[Document(metadata={'title': 'Introduction', 'header': 'Not available', 'url': 'https://people.iitism.ac.in/~download/MalviyaMission/MMTTC%20-%20IIT(ISM)%20Dhanbad.pdf \n https://www.iitism.ac.in/center'}, page_content=' Title-Introduction , heading-Not available , url-https://people.iitism.ac.in/~download/MalviyaMission/MMTTC%20-%20IIT(ISM)%20Dhanbad.pdf \n https://www.iitism.ac.in/center \n\n description - • The National Education Policy (NEP) 2020 underscores the need for motivated, energized, and capable faculty in Higher Education. The capacity building for teachers at all levels is a key focus.'),
 Document(metadata={'title': 'Introduction', 'header': 'Not available', 'url': 'https://people.iitism.ac.in/~download/MalviyaMission/MMTTC%20-%20IIT(ISM)%20Dhanbad.pdf \n https://www.iitism.ac.in/center'}, page_content=' Title-Introduction , heading-Not available , url-https://people.iitism.ac.in/~download/MalviyaMission/MMTTC%20-%20IIT(ISM)%20Dhanbad.pdf \n https://www.iitism.ac.in/cen

In [17]:

vector_store.add_documents(structured_docs)
vector_store.persist()

/tmp/ipython-input-1598627614.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [18]:
vector_store.similarity_search(
    query="national education policy",
    k=10
)

[Document(metadata={'header': 'Not available', 'url': 'https://people.iitism.ac.in/~download/MalviyaMission/MMTTC%20-%20IIT(ISM)%20Dhanbad.pdf \n https://www.iitism.ac.in/center', 'title': 'Introduction'}, page_content=' Title-Introduction , heading-Not available , url-https://people.iitism.ac.in/~download/MalviyaMission/MMTTC%20-%20IIT(ISM)%20Dhanbad.pdf \n https://www.iitism.ac.in/center \n\n description - • The National Education Policy (NEP) 2020 underscores the need for motivated, energized, and capable faculty in Higher Education. The capacity building for teachers at all levels is a key focus.'),
 Document(metadata={'title': 'Introduction', 'header': 'Not available', 'url': 'https://people.iitism.ac.in/~download/MalviyaMission/MMTTC%20-%20IIT(ISM)%20Dhanbad.pdf \n https://www.iitism.ac.in/center'}, page_content=' Title-Introduction , heading-Not available , url-https://people.iitism.ac.in/~download/MalviyaMission/MMTTC%20-%20IIT(ISM)%20Dhanbad.pdf \n https://www.iitism.ac.in/cen

# download db


In [19]:
import shutil
from google.colab import files

# Zip the folder
shutil.make_archive("chroma_db", 'zip', "chroma_db")

# Download the zip
files.download("chroma_db.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>